<a href="https://colab.research.google.com/github/paramrajyadav/Chat_with_PDF_FAISS/blob/main/Chat_with_PDF_FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# !pip install langchain openai tiktoken rapidocr-onnxruntime
# !pip install faiss-cpu
# !pip install langchain-community
# !pip install PyPDF2
# !pip install unstructured[pdf]
# !pip install langchain
# !pip install unstructured
# !pip install openai
# !pip install Cython
# !pip install tiktoken

# !pip install --upgrade langchain-astradb

In [5]:
# import requests
# from langchain.document_loaders import TextLoader
# from langchain.vectorstores import FAISS

# import os
# from getpass import getpass

# #     load_dataset,
# # )from datasets import (

# from langchain_community.document_loaders import PyPDFLoader
# from langchain_core.documents import Document
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.runnables import RunnablePassthrough

# from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain.document_loaders import UnstructuredPDFLoader
# from langchain.indexes import VectorstoreIndexCreator

In [6]:

from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [7]:

import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [8]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [9]:
pdf_folder_path="/content/drive/MyDrive/Dataset/text"

In [10]:
os.listdir(pdf_folder_path)

['05-07-fintech-space-continues-to-be-competitive-and-drama-filled.txt',
 '05-07-3one4-capital-driven-by-contrarian-bets-raises-200-million-new-fund.txt',
 '05-06-this-week-in-apps-apple-and-google-team-up-on-trackers-google-i-o-preview-apps-hit-newfronts.txt',
 '05-06-ai-startups-q1-investments.txt',
 '05-07-spacex-starship-startups-future.txt',
 '05-06-amazon-launches-free-channels-check-marks-come-to-gmail-and-openai-raises-more-moolah.txt',
 '05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt',
 '05-04-slack-updates-aim-to-put-ai-at-the-center-of-the-user-experience.txt',
 '05-04-hugging-face-and-servicenow-release-a-free-code-generating-model.txt',
 '05-05-vint-cerf-on-the-exhilarating-mix-of-thrill-and-hazard-at-the-frontiers-of-tech.txt',
 '05-05-google-and-openai-are-walmarts-besieged-by-fruit-stands.txt',
 '05-05-google-i-o-2023-is-next-week-heres-what-were-expecting.txt',
 '05-05-with-deepfloyd-generative-ai-art-gets-a-text-upgrade.txt',
 '05-04-cma-generati

In [19]:
pdf_file_path=os.path.join(pdf_folder_path,"state_of_the_union.txt")

In [20]:

# loader1=TextLoader(pdf_file_path)

In [21]:
# document=loader1.load()

In [22]:
# print(document[0].page_content)

In [23]:

# from langchain.text_splitter import RecursiveCharacterTextSplitter

In [24]:
# text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)

In [25]:
# text_chunks=text_splitter.split_documents(document)

In [26]:
# text_chunks

In [27]:

# from langchain.embeddings import OpenAIEmbeddings
# from langchain.vectorstores import FAISS

In [28]:
# from google.colab import userdata
# OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [33]:

# import os
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [34]:

embeddings=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

NameError: name 'OpenAIEmbeddings' is not defined

In [35]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)

NameError: name 'FAISS' is not defined

In [36]:
retriever=vectorstore.as_retriever()

NameError: name 'vectorstore' is not defined

In [37]:

from langchain.prompts import ChatPromptTemplate

ModuleNotFoundError: No module named 'langchain'

In [38]:
template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [39]:
prompt=ChatPromptTemplate.from_template(template)

NameError: name 'ChatPromptTemplate' is not defined

In [ ]:

from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [ ]:
output_parser=StrOutputParser()

In [40]:

llm_model=ChatOpenAI(openai_api_key=OPENAI_API_KEY,model_name="gpt-3.5-turbo")

NameError: name 'ChatOpenAI' is not defined

In [ ]:

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)


In [ ]:
rag_chain.invoke("How is the United States supporting Ukraine economically and militarily?")